<a href="https://colab.research.google.com/github/Zenith1618/LLM/blob/main/Sentiment_Analysis_Using_RoBERTa_on_custom_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
# Install the transformers library
!pip install datasets transformers accelerate

In [ ]:
!pip install transformers[torch]

In [ ]:
# Import required packages
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
from transformers import pipeline
import accelerate

In [ ]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Use RoBERTa Tokenizer and RoBERTa pre-trained models

### Load the pre-trained RoBERTa model and its tokenizer.



https://huggingface.co/google-bert/bert-large-uncased-whole-word-masking-finetuned-squad

In [ ]:
model_name = "siebert/sentiment-roberta-large-english"  #Model is pretrained on squad
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


### Dataset Preparation

*   Text to Tokens
*   Tokens to Unique IDS
*   Map Tokens with its unique IDs in a dictionary


In [ ]:
class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts

    def __len__(self):
        return len(self.tokenized_texts["input_ids"])

    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}

### Tokenize the user input text

In [ ]:
tweets = [
"Just had the best meal ever at my favorite restaurant! The food was amazing, and the service was top-notch. Feeling so satisfied and happy right now! 😃🍔🥗 #Foodie #HappyCustomer",
"Feeling really frustrated after sitting in traffic for hours today. Missed an important meeting and wasted so much time. Not a great start to the day. 😤🚗 #TrafficWoes #Stressed"
]

In [ ]:
tokenized_texts = tokenizer(tweets,truncation=True,padding=True)
pred_dataset = SimpleDataset(tokenized_texts)

### Model

In [ ]:
trainer = Trainer(model=model)

In [ ]:
predictions = trainer.predict(pred_dataset) #run predictions

In [ ]:
preds = predictions.predictions.argmax(-1) #get prediction labels
labels = pd.Series(preds).map(model.config.id2label) #convert prediction into POSITIVE or NEGATIVE
scores = (np.exp(predictions[0])/np.exp(predictions[0]).sum(-1,keepdims=True)).max(1) # ACCURACY SCORE

In [ ]:
scores

array([0.9989399 , 0.99949604], dtype=float32)

In [ ]:
labels

0    POSITIVE
1    NEGATIVE
dtype: object

### RoBERTa Sentiment results on user input

In [ ]:
df = pd.DataFrame(list(zip(tweets,preds,labels,scores)), columns=['text','pred','label','score'])
df.head()

,text,pred,label,score
0,Just had the best meal ever at my favorite res...,1,POSITIVE,0.998940
1,Feeling really frustrated after sitting in tra...,0,NEGATIVE,0.999496
